In [142]:
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2
import pandas as pd
import pickle as pkl
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from notebook_utils import *
from parameter_estimation import load_data_util
import numpy as np
import pandas as pd
import snap
import operator
import re
from sklearn import metrics
import networkx as nx
import scikitplot as skplt
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='serif', size='13') # controls default text sizes
plt.rc('xtick', labelsize='12')
plt.rc('ytick', labelsize='12')
plt.rc('legend', fontsize='13') # legend fontsize
plt.rc('axes', labelsize='12', titlesize='13')  # fontsize of the x and y labels # fontsize of the axes title
# plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read the cascade from original files

In [102]:
import os, sys
import time, datetime
import numpy as np
from email.utils import parsedate_tz, mktime_tz


def getCascadeSequences_and_SeqLabel():
    
    #-----------------------------
    
    # savepath="./data/kwon/"
    # path="kwon/"

    # savepath="heme-rnr-dataset/"
    # path="pheme-rnr-dataset/"

#     savepath="twitter-ma/"
    path="twitter-ma/"

    # savepath="twitter-qian/"
    # path="twitter-qian/"

    #-----------------------------
    
    cascades = "Tweets/" # uid, tid, content, time (each file is a diffusion cascade)
    
    #----------------------------- GET (uid, time)

    diff_cascades, str_d = {}, {}
    labels = []

    for tweetsfile in os.listdir(path+cascades):
        f = open(path+cascades+tweetsfile, "r")
        # print tweetsfile
        print(tweetsfile)
        ulist, tslist = list(), list()
        for line in f.readlines():
            # break 
            # TMA format
            uid, timestamp = line.split("\t")[0], int(line.split("\t")[-1])
            # print(uid, timestamp)
            
#             # kwon format:            
#             uid, timestamp = line.split("\t")[0], time.mktime(datetime.datetime.strptime(
#                 line.split("\t")[-1].rstrip(), '%Y-%m-%d %H:%M:%S').timetuple())
            
#             # pheme format
#             uid, timestamp = line.split("\t")[0], mktime_tz(parsedate_tz(line.split("\t")[-1].rstrip()))

            ulist.append(uid)
            tslist.append(timestamp)
        tslist, ulist = zip(*sorted(zip(tslist, ulist)))
        rel_tslist = np.array(tslist) - np.array(tslist[0])
        if len(ulist) <= 1: continue
        u = np.array(ulist, dtype=np.int64)
        t = np.array(rel_tslist)
        cas = np.vstack([u, t]).transpose()
        # print(cas)
        u10str = ""
        for i in cas[0:10, 0]:
            u10str += str(int(i))
        str_d[u10str] = tweetsfile
        
        diff_cascades[tweetsfile] = cas  #(ulist, rel_tslist)
        
        # break
    #----------------------------- Write (uid, time, ...) and labels
    return str_d, diff_cascades


In [103]:
d, _ = getCascadeSequences_and_SeqLabel()

N_E794.txt
N_TM1728.txt
R_E597.txt
R_E489.txt
N_E772.txt
R_E168.txt
R_E152.txt
R_E422.txt
N_TM80.txt
N_TM787.txt
N_TM2525.txt
N_E781.txt
N_TM1852.txt
N_E373.txt
N_TM1623.txt
N_TM555.txt
R_E574.txt
N_TM1706.txt
N_E510.txt
R_E138.txt
N_TM527.txt
N_E520.txt
N_E759.txt
N_Havard.txt
N_E707.txt
R_E310.txt
R_E586.txt
N_TM840.txt
R_E47.txt
N_TM392.txt
R_E46.txt
R_E390.txt
N_E761.txt
R_E121.txt
N_E23.txt
N_E602.txt
R_E259.txt
N_E437.txt
N_JenniferHudson.txt
N_E675.txt
R_E658.txt
R_E726.txt
R_E633.txt
R_E656.txt
R_E314.txt
R_E273.txt
N_E354.txt
N_ObamaFishing.txt
N_TM982.txt
R_E735.txt
R_E151.txt
R_E733.txt
R_E702.txt
R_E68.txt
N_TM1315.txt
R_E389.txt
R_E262.txt
R_E93.txt
N_E514.txt
N_E363.txt
R_E706.txt
N_E784.txt
R_E127.txt
N_TM2108.txt
R_E211.txt
N_TM453.txt
R_E642.txt
N_E682.txt
N_TM1535.txt
R_E337.txt
N_E644.txt
N_E548.txt
R_E57.txt
N_E153.txt
R_E376.txt
R_E719.txt
R_E227.txt
N_E467.txt
R_E51.txt
R_E700.txt
R_E607.txt
N_E270.txt
N_E596.txt
N_TM2156.txt
R_E214.txt
N_TM1747.txt
R_E410.txt
N_T

In [105]:
print(len(d))

990


In [106]:
# cascade mapping to names
cascades = read_cascades_file('../data/tma/cascades.txt')
print(len(cascades))

cascade_names = []

# find name for the cascade from dict.
for cas in cascades:
    u10str = ""
    for i in cas[0:10, 0]:
        u10str += str(int(i))
    # print('str', u10str)
    if u10str in d:
        # print(u10str, d[u10str])
        filename = d[u10str]
        cascade_names.append(filename)
        
print(len(cascade_names))
# print(cascade_names)

992
992


In [107]:
np.savetxt('../data/tma/cascade_names.txt', cascade_names, fmt='%s')

# Find stanceeval baseline preds 

In [152]:
def stanceeval_get_pred_fake_cascades():
    # data="kwon/sentiments.txt"
    data="twitter-ma/twitter-ma.csv"
    df = pd.read_csv(data)
    print(df.head())
    senti = df.groupby(['fact'], as_index=False).mean()
    # create cascades out of this
    preds = (senti['sentiment_score']>0)*1
    event_names = list(senti['fact'])
    ground = []
    for row in senti['fact']:
        if row.split("_")[0] == 'N':
            ground.append(0)
        else:
            ground.append(1)
    # print(ground)
    print(metrics.classification_report(ground, preds))
    
    # print(event_names)
    dict_preds = dict(zip(event_names, preds))
    
    # read cascade_names file to reorder the predictions accordingly.
    
    # casnames = np.loadtxt('../data/kwon/cascade_names.txt', dtype=np.str)
    casnames = np.loadtxt('../data/tma/cascade_names.txt', dtype=np.str)
    # print(list(casnames)) 

    ordered_preds = []
    for name in casnames:
        key = re.sub('.txt$', '', name)
        # key = name.strip(".txt\n")
        if key not in dict_preds:
            print('name', name, key)
        else:
            ordered_preds.append(dict_preds[key])
        # break
    
    return ordered_preds

In [153]:
preds = stanceeval_get_pred_fake_cascades()
np.savetxt('../output/baselines/tma/stanceeval_pred_results.txt', preds, fmt='%d')
# top_fake_users = list(get_topK_fake_users_from_pred_fake_cascades(preds, train_cascades))
    # return top_fake_users

      website    fact  object  sentiment_score
0  1003856029  N_E312  N_E312          -0.6597
1  1013430282  N_E312  N_E312          -0.7149
2  1014445069  N_E312  N_E312          -0.7149
3   101596871  N_E312  N_E312           0.5267
4    10228272  N_E312  N_E312          -0.7412
              precision    recall  f1-score   support

           0       0.47      0.61      0.53       494
           1       0.46      0.34      0.39       498

    accuracy                           0.47       992
   macro avg       0.47      0.47      0.46       992
weighted avg       0.47      0.47      0.46       992

